# Project Set-up


In [3]:
try: 
    import pandas as pd
    import ta
    import numpy as np
    import requests
    import sys, os
    from dotenv import load_dotenv
    from pathlib import Path
    from datetime import datetime, timedelta
    
    print("Successfully Imported all the libraries")

except Exception as e: 
    print(f"Import Error: {e}")
    raise


Successfully Imported all the libraries


In [2]:
# PROJECT_ROOT = Path('/Users/shengyaotang/Desktop/Quant Project/ML_Portfolio_Management')
PROJECT_ROOT = Path('..')
DATA_DIR = PROJECT_ROOT/'data'

load_dotenv(PROJECT_ROOT/'.env')
print(".env file successfully loaded")


.env file successfully loaded


# Data Extraction: 
Extract S&P 500 healthcare data from wikipedia and financial modeling Prep Api

### Get Ticker data

In [3]:

alpha_vantage_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
etf_symbol = "DIA"

def get_qqq_holdings(api_key: str, symbol: str) -> pd.DataFrame:

    url = f'https://www.alphavantage.co/query?function=ETF_PROFILE&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200: 
        json_data = response.json()
        holdings_data = json_data["holdings"]
        df = pd.DataFrame(holdings_data)
        rows_to_drop = df[df['symbol'] == 'n/a'].index
        df = df.drop(rows_to_drop, axis = 0)

    else: 
        print("API request to Alpha Vantage failed.")
    df.to_csv(DATA_DIR/"raw"/"holdings.csv", index = False)
    # df.to_csv('../data/raw/holdings.csv', index = False)
    return df


In [4]:
df = get_qqq_holdings(alpha_vantage_api_key, etf_symbol)
holdings_list = df['symbol'].to_list()
print(holdings_list)

['GS', 'CAT', 'MSFT', 'AXP', 'V', 'HD', 'UNH', 'AMGN', 'SHW', 'JPM', 'MCD', 'IBM', 'TRV', 'AAPL', 'CRM', 'AMZN', 'BA', 'JNJ', 'HON', 'NVDA', 'MMM', 'CVX', 'PG', 'DIS', 'WMT', 'MRK', 'CSCO', 'KO', 'NKE', 'VZ']


### Extract historical price data from financial modeling prep

In [5]:
base_url = "https://financialmodelingprep.com"
data_type = "historical-price-eod"
ticker = "AAPL"
FMP_API_KEY = os.getenv("FMP_API_KEY")

url = f"{base_url}/stable/{data_type}/full?symbol={ticker}&apikey={FMP_API_KEY}"


In [6]:
def get_historical_price(url: str) -> pd.DataFrame: 
    try: 
        response = requests.get(url)
        print(response.status_code)
        data = response.json()
        df = pd.DataFrame(data, index = range(len(data)))
        print("Got historical price successfully")
    except Exception as e:
        # print(f"There is an error: {e}")
        df = pd.DataFrame()
        
    return df

In [7]:
def get_historical_prices(tickers: list, api_key: str) -> pd.DataFrame:

    df = pd.DataFrame()
    for ticker in tickers:
        url = f"{base_url}/stable/{data_type}/full?symbol={ticker}&apikey={api_key}"
        temp_df = get_historical_price(url)
        df = pd.concat([df,temp_df], axis = 0)

    # df.to_csv("/Users/shengyaotang/Desktop/Quant Project/ML_Portfolio_Management/data/raw/historical_price.csv", index = False)
    df.to_csv(DATA_DIR/"raw"/"historical_price.csv", index = False)
    # df.to_csv('../data/raw/historical_price.csv', index = False)
    return df

In [8]:
df = get_historical_prices(holdings_list, FMP_API_KEY)

200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
402
402
200
Got historical price successfully
402
402
402
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully


In [9]:
df.head(10)
df_holdings = pd.DataFrame(df["symbol"].unique(), columns = ["symbol"])
df_holdings.to_csv(DATA_DIR/"raw"/"holdings.csv", index = False)



# Data Transformation

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

### Import the dataset

In [11]:
def load_data(filename): 
    df = pd.read_csv(filename)
    return df

In [12]:
# df = read_data(DATA_DIR)
df = load_data(DATA_DIR/"raw"/"historical_price.csv")
df.head(10)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap
0,GS,2026-01-07,956.88,958.25,934.00,941.02,2680846,-15.86,-1.65747,944.4200
1,GS,2026-01-06,949.41,958.57,943.25,955.47,1966000,6.06,0.63829,951.6750
2,GS,2026-01-05,914.40,961.69,912.60,948.44,3742022,34.04,3.72000,934.2825
3,GS,2026-01-02,884.00,914.44,880.75,914.34,2817613,30.34,3.43000,898.3825
4,GS,2025-12-31,884.10,886.00,876.79,879.00,1221740,-5.10,-0.57686,881.4725
5,GS,2025-12-30,894.74,895.02,881.18,884.42,1833300,-10.32,-1.15000,888.8400
6,GS,2025-12-29,906.45,906.48,891.56,892.18,1578000,-14.27,-1.57000,899.1675
7,GS,2025-12-26,911.00,913.32,905.31,907.04,995328,-3.96,-0.43469,909.1675
8,GS,2025-12-24,901.16,911.88,898.70,910.78,771800,9.62,1.07000,905.6300
9,GS,2025-12-23,900.35,905.92,893.70,901.71,1210438,1.37,0.15105,900.4200


### Create Target Variable
1. Calculate weekly log returns

In [13]:
# Calculate weekly returns:
def calculate_weekly_returns(df: pd.DataFrame) -> pd.DataFrame:
    df['weekly_return'] = df.groupby('symbol')['close'].pct_change(periods = -5)
    df = df.dropna(subset = ["weekly_return"])
    
    df["weekly_log_return"] = np.log(1 + df["weekly_return"])
    
    return df

In [14]:
df = calculate_weekly_returns(df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1573/387843821.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weekly_log_return"] = np.log(1 + df["weekly_return"])


,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return
0,GS,2026-01-07,956.88,958.25,934.00,941.02,2680846,-15.86,-1.65747,944.4200,0.063997,0.062032
1,GS,2026-01-06,949.41,958.57,943.25,955.47,1966000,6.06,0.63829,951.6750,0.070939,0.068535
2,GS,2026-01-05,914.40,961.69,912.60,948.44,3742022,34.04,3.72000,934.2825,0.045643,0.044632


2. 🎯 Create Target Variable

In [15]:
def create_target_variable(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    
    median_returns = df.groupby('year_week')['weekly_log_return'].median().reset_index()
    median_returns.columns = ['year_week', 'median_weekly_return']
    
    # Merge median returns back to original dataframe
    if 'median_weekly_return' not in df.columns:
        df = pd.merge(df, median_returns, on='year_week', how='left')
        
    df['target'] = (df['weekly_log_return'] > df['median_weekly_return']).astype(int)
    
    return df

In [16]:
df = create_target_variable(df)
df.head(3)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return,year_week,median_weekly_return,target
0,GS,2026-01-07,956.88,958.25,934.00,941.02,2680846,-15.86,-1.65747,944.4200,0.063997,0.062032,2026-01,0.006371,1
1,GS,2026-01-06,949.41,958.57,943.25,955.47,1966000,6.06,0.63829,951.6750,0.070939,0.068535,2026-01,0.006371,1
2,GS,2026-01-05,914.40,961.69,912.60,948.44,3742022,34.04,3.72000,934.2825,0.045643,0.044632,2026-01,0.006371,1


In [17]:
# 🗂 Create the panel dataset indexed by date and tickers

def create_panel_dataset(df: pd.DataFrame) -> pd.DataFrame:
    return df.sort_values(by = "date")

In [18]:
# This step could be moved to the pipeline
df = create_panel_dataset(df)
df.head(3)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return,year_week,median_weekly_return,target
21232,VZ,2021-01-19,57.44,57.58,56.98,57.05,18821300,-0.39,-0.67897,57.2625,-0.007135,-0.007161,2021-03,-0.007677,1
1248,GS,2021-01-19,305.00,306.60,293.80,294.20,6727800,-10.80,-3.54000,299.9000,0.001259,0.001258,2021-03,-0.007677,1
18734,KO,2021-01-19,48.74,49.04,48.24,48.51,29114500,-0.23,-0.47189,48.6325,-0.033665,-0.034245,2021-03,-0.007677,0


In [19]:
# 🗑 Remove columns that will not be used for the project
def remove_columns(cols: list, df: pd.DataFrame) -> pd.DataFrame:
    try: 
        df = df.drop(columns = cols, axis = 1)
    except Exception as e:
        print(e)
    finally: 
        return df

In [20]:
cols = ["open", "high", "low", "change", 
        "changePercent", "weekly_return", 
       "year_week", "median_weekly_return"]
df = remove_columns(cols, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target
21232,VZ,2021-01-19,57.05,18821300,57.2625,-0.007161,1
1248,GS,2021-01-19,294.20,6727800,299.9000,0.001258,1
18734,KO,2021-01-19,48.51,29114500,48.6325,-0.034245,0


2. Calculate moving averages

In [21]:
#📈 Moving Averages

def calc_moving_avg(window: int, df: pd.DataFrame) -> pd.DataFrame:
    col_name = f'MA_{str(window)}'
    df[col_name] = df.groupby('symbol')['close'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).mean()
    )
    
    return df


In [22]:
def calc_moving_avgs(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        df = calc_moving_avg(window, df)
    return df

In [23]:
windows = [200, 100, 50]
df = calc_moving_avgs(windows, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50
21232,VZ,2021-01-19,57.05,18821300,57.2625,-0.007161,1,57.05,57.05,57.05
1248,GS,2021-01-19,294.20,6727800,299.9000,0.001258,1,294.20,294.20,294.20
18734,KO,2021-01-19,48.51,29114500,48.6325,-0.034245,0,48.51,48.51,48.51


### Risk and Volatility

In [24]:
def calc_volatility(col_name: str, window: int, df: pd.DataFrame) -> pd.DataFrame: 
    df[col_name] = df.groupby('symbol')['weekly_log_return'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).std() * np.sqrt(window)
    )
    
    return df

In [25]:
def calc_volatilties(volatility_dict: dict, df: pd.DataFrame) -> pd.DataFrame: 
    for col_name, window in volatility_dict.items():
        df = calc_volatility(col_name, window, df)
    return df

In [26]:
volatility_dict = {
    'vol_1M': 4,    # 1 month ≈ 4 weeks
    'vol_6M': 26,   # 6 months ≈ 26 weeks
    'vol_12M': 52   # 12 months ≈ 52 weeks
}

df = calc_volatilties(volatility_dict, df)
#df = df.dropna(subset = ['vol_1M', 'vol_6M','vol_12M'])

df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M
21232,VZ,2021-01-19,57.05,18821300,57.2625,-0.007161,1,57.05,57.05,57.05,NaN,NaN,NaN
1248,GS,2021-01-19,294.20,6727800,299.9000,0.001258,1,294.20,294.20,294.20,NaN,NaN,NaN
18734,KO,2021-01-19,48.51,29114500,48.6325,-0.034245,0,48.51,48.51,48.51,NaN,NaN,NaN


### Short Term Reversal Factors

In [27]:
def calculate_rsi(window, group): 
    rsi = RSIIndicator(close = group['close'], window = window).rsi()
    return rsi

In [28]:
def calculate_rsis(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        col_name = f'RSI_{str(window)}'
        group = df.groupby('symbol', group_keys = False)
        df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
        
    return df

In [29]:
windows = [3, 9, 14]
df = calculate_rsis(windows, df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1573/1543531198.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1573/1543531198.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
/var/folders/gl/wt1cfyrs16358nbd8jfg3k70

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14
21232,VZ,2021-01-19,57.05,18821300,57.2625,-0.007161,1,57.05,57.05,57.05,NaN,NaN,NaN,NaN,NaN,NaN
1248,GS,2021-01-19,294.20,6727800,299.9000,0.001258,1,294.20,294.20,294.20,NaN,NaN,NaN,NaN,NaN,NaN
18734,KO,2021-01-19,48.51,29114500,48.6325,-0.034245,0,48.51,48.51,48.51,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
def calculate_bb(group, window = 20, window_dev = 2):
    return BollingerBands(close = group['close'], window = window, window_dev = window_dev)

In [31]:
def calculate_bbs(bands: list, df:pd.DataFrame) -> pd.DataFrame:
    group = df.groupby('symbol', group_keys = False)
    for band in bands:
        if band == "hband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
        elif band == "lband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())
            
    return df

In [32]:
bands = ["hband", "lband"]
df = calculate_bbs(bands, df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1573/3629412943.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1573/3629412943.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())


,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband
21232,VZ,2021-01-19,57.05,18821300,57.2625,-0.007161,1,57.05,57.05,57.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1248,GS,2021-01-19,294.20,6727800,299.9000,0.001258,1,294.20,294.20,294.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18734,KO,2021-01-19,48.51,29114500,48.6325,-0.034245,0,48.51,48.51,48.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Momentum Factor: 

In [33]:
def calculate_momentum(window_months: int, df: pd.DataFrame, trading_days :int = 21) -> pd.DataFrame:
    window_days = window_months * trading_days
    col_name = f"momentum_{str(window_months)}M"
    df[col_name] = df.groupby('symbol')['close'].pct_change(periods = window_days)
    
    return df

In [34]:
def calculate_momentums(windows: list, df:pd.DataFrame) -> pd.DataFrame:
    for window in windows:
        df = calculate_momentum(window, df)
    return df

In [35]:
windows_for_momentum = [12, 6, 1]

df = calculate_momentums(windows_for_momentum, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
21232,VZ,2021-01-19,57.05,18821300,57.2625,-0.007161,1,57.05,57.05,57.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1248,GS,2021-01-19,294.20,6727800,299.9000,0.001258,1,294.20,294.20,294.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18734,KO,2021-01-19,48.51,29114500,48.6325,-0.034245,0,48.51,48.51,48.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter Datasets:

In [36]:
# 🗓Choose Each Wednesday and Thursday as the data transformation
def wed_thurs_selector(df, date_col='date', stock_col='symbol'):

    df['year_week'] = df[date_col].dt.strftime('%Y-%U')
    df['day_of_week'] = df[date_col].dt.dayofweek
    
    
    wed_thu = df[df['day_of_week'].isin([2, 3])].copy()
    
    
    filtered = wed_thu.groupby([stock_col, 'year_week']).first().reset_index()
    
    return filtered.drop(columns=['day_of_week', 'year_week'])
    

In [37]:
def filter_data(df: pd.DataFrame) -> pd.DataFrame: 
    df = df.copy()
    df = wed_thurs_selector(df)
    return df.dropna(ignore_index = True)

In [38]:
df = filter_data(df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
0,AAPL,2022-01-19,166.23,94815000,168.3125,-0.051871,0,147.0463,157.6724,167.7942,...,0.164759,0.257659,11.730773,31.598820,39.999691,182.973032,167.483968,0.259032,0.143260,-0.028690
1,AAPL,2022-01-26,159.69,108275308,161.3500,-0.040138,0,147.8017,158.1554,169.0138,...,0.179705,0.298390,3.345620,22.148664,32.126858,184.839828,157.965172,0.124102,0.101462,-0.114457
2,AAPL,2022-02-02,175.84,84914300,174.9500,0.096340,1,148.7324,159.0118,170.6512,...,0.245340,0.330057,90.620084,65.943436,59.887270,180.925805,157.671195,0.312827,0.196597,-0.033899


In [39]:
def save_processed_data(df: pd.DataFrame, directory): 
    df.to_csv(directory/"processed"/"processed_historical_price.csv", index = False)

In [40]:
save_processed_data(df, DATA_DIR)

### Create Custom transformer
E.g. Remove/Fill in Nan values, log transformation, standardization

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

In [42]:
# Perform Log Transformation: 
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.features:
            X[feature] = np.log(X['close'] / X[feature])
        return X

### 💽Create training and testing variable

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold

import joblib

In [44]:
# Create x and y variable

def create_variables(df: pd.DataFrame) -> list: 
    return [df.drop(columns = ["target"]), df["target"]]

X, y = create_variables(df)


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [46]:
# symbols = df["symbol"].unique().tolist()

# X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
# for symbol in symbols: 
#     symbol_df = df[df["symbol"] == symbol]
#     cut = int(len(symbol_df) * 0.8)
#     Xs, ys = create_variables(symbol_df)
#     Xs_train, Xs_test = Xs[:cut], Xs[cut:]
#     ys_train, ys_test = ys[:cut], ys[cut:]

#     X_train = pd.concat([X_train, Xs_train])
#     X_test = pd.concat([X_test, Xs_test])
#     y_train = pd.concat([y_train, ys_train])
#     y_test = pd.concat([y_test, ys_test])


def time_aware_split(df): 
    symbols = df["symbol"].unique().tolist()

    X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for symbol in symbols:
        symbol_df = df[df["symbol"] == symbol]
        cut = int(len(symbol_df) * 0.8)
        Xs, ys = create_variables(symbol_df)
        Xs_train, Xs_test = Xs[:cut], Xs[cut:]
        ys_train, ys_test = ys[:cut], ys[cut:]

        X_train = pd.concat([X_train, Xs_train])
        X_test = pd.concat([X_test, Xs_test])
        y_train = pd.concat([y_train, ys_train])
        y_test = pd.concat([y_test, ys_test])

    return X_train, X_test, y_train, y_test



In [47]:
X_train, X_test, y_train, y_test = time_aware_split(df)

In [48]:
# tscv = TimeSeriesSplit(n_splits=3)
# for i, (train_index, test_index) in enumerate(tscv.split(df)):
#     print(f"Fold {i + 1}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

### 🏭 Build a pipeline to transform features

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [50]:
#build pipeline
log_transform_features = ['MA_200', 'MA_100', 'MA_50', 'hband', 'lband']
# cols_to_drop = ['Ticker', 'date', 'weekly_log_return']

cols_to_drop = ['symbol', 'date', 'weekly_log_return']

# Example: Drop columns 'col1' and 'col2'
col_dropper = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', cols_to_drop)
    ],
    remainder='passthrough'  # keeps all other columns
)

logistic_regression_pipeline = Pipeline(steps = [ 
    ('log_transformers', LogTransformer(log_transform_features)),
    ('col_dropper', col_dropper), 
    ('scaler', StandardScaler()), 
    ('classifier', LogisticRegression())
])

In [51]:
logistic_regression_model = logistic_regression_pipeline.fit(X_train, y_train)

/Users/shengyaotang/opt/anaconda3/envs/bootcamp_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
filename = "logistic_regression_model.sav"
joblib.dump(logistic_regression_model, open(filename, 'wb'))

In [53]:
logistic_regression_model = joblib.load(open(filename, 'rb'))
y_pred = logistic_regression_model.predict(X_test)
print(y_pred)

[1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1.
 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0.
 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1.

In [54]:
print(f"Model accuracy: {logistic_regression_model.score(X_test, y_test)}")

Model accuracy: 0.7633053221288515


# Create Weekly Stock Portfolio

In [55]:
def get_last_week_data(filename):
    df = load_data(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    last_year_week = (datetime.now() - timedelta(weeks = 1)).strftime('%Y-%U')

    df_last_week = df[df['year_week'] == last_year_week]
    return df_last_week.drop(columns=['year_week'])

In [56]:
filename = DATA_DIR/'processed'/'processed_historical_price.csv'
df_last_week = get_last_week_data(filename).reset_index(drop = True)
X_last_week = df_last_week.drop(columns = ['target'])
y_last_week = df_last_week['target']


df_last_week

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M


In [57]:
y_pred = logistic_regression_model.predict(X_last_week)
y_prob = logistic_regression_model.predict_proba(X_last_week)

df_pred = pd.DataFrame(y_pred, columns = ['prediction'])
df_prob = pd.DataFrame(y_prob, columns = ['underperform', 'outperform'])

df_pred

ValueError: Found array with 0 sample(s) (shape=(0, 17)) while a minimum of 1 is required by StandardScaler.

In [ ]:
def create_weekly_stock_portfolio(*args: pd.DataFrame) -> pd.DataFrame:
    df_stock_portfolio = pd.concat(args, axis = 1)
    df_stock_portfolio = df_stock_portfolio.sort_values(by = ['outperform'], ascending = False)
    df_stock_portfolio = df_stock_portfolio[df_stock_portfolio['prediction'] == 1]
    df_stock_portfolio['weight'] = 1 / len(df_stock_portfolio)
    df_stock_portfolio.to_csv(DATA_DIR/'processed'/'stock_portfolio.csv', index = False)
    return df_stock_portfolio.reset_index(drop = True)

In [ ]:
df_stock_portfolio = create_weekly_stock_portfolio(df_last_week['symbol'], df_prob, df_pred)
df_stock_portfolio

,symbol,underperform,outperform,prediction,weight
0,NKE,0.063142,0.936858,1.0,0.2
1,CVX,0.129739,0.870261,1.0,0.2
2,VZ,0.179859,0.820141,1.0,0.2
3,BA,0.443586,0.556414,1.0,0.2
4,NVDA,0.449738,0.550262,1.0,0.2


In [ ]:
def update_stock_portfolio(df_stock_portfolio: pd.DataFrame, df_historical_price: pd.DataFrame) -> pd.DataFrame:
    df_historical_price = df_historical_price.copy()
    
    df_historical_price['date'] = pd.to_datetime(df_historical_price['date'])
    df_historical_price['week_of_year'] = df_historical_price['date'].dt.strftime('%Y-%U')
    df_weekly_price = df_historical_price[df_historical_price['week_of_year'] == datetime.now().strftime('%Y-%U') ]

    if 'close' not in df_stock_portfolio.columns: 
        df_weekly_portfolio = pd.merge(df_stock_portfolio, df_weekly_price, on = 'symbol', how = 'left')
    df_weekly_portfolio['week_of_day'] = df_weekly_portfolio['date'].dt.weekday
    df_weekly_portfolio.to_csv(DATA_DIR/'processed'/'weekly_portfolio.csv')

    return df_weekly_portfolio.dropna()

In [ ]:
df_historical_price = load_data(DATA_DIR/'raw'/'historical_price.csv')
df_weekly_portfolio = update_stock_portfolio(df_stock_portfolio, df_historical_price)
df_weekly_portfolio

,symbol,underperform,outperform,prediction,weight,date,open,high,low,close,volume,change,changePercent,vwap,week_of_year,week_of_day
0,NKE,0.063142,0.936858,1.0,0.2,2026-01-06,64.06,65.43,63.41,65.35,19973192,1.290,2.010000,64.5625,2026-01,1
1,NKE,0.063142,0.936858,1.0,0.2,2026-01-05,63.74,65.37,63.39,64.53,23482539,0.795,1.240000,64.2575,2026-01,0
2,CVX,0.129739,0.870261,1.0,0.2,2026-01-06,164.88,165.05,156.11,156.54,20272783,-8.340,-5.060000,160.6450,2026-01,1
3,CVX,0.129739,0.870261,1.0,0.2,2026-01-05,165.75,165.75,159.31,163.85,35094900,-1.900,-1.150000,163.6650,2026-01,0
4,VZ,0.179859,0.820141,1.0,0.2,2026-01-06,40.34,40.46,39.99,40.30,32758409,-0.040,-0.099157,40.2725,2026-01,1
5,VZ,0.179859,0.820141,1.0,0.2,2026-01-05,40.43,40.52,39.83,40.23,43091710,-0.200,-0.494680,40.2525,2026-01,0
6,BA,0.443586,0.556414,1.0,0.2,2026-01-06,229.23,231.64,228.60,229.84,6714333,0.615,0.266110,229.8275,2026-01,1
7,BA,0.443586,0.556414,1.0,0.2,2026-01-05,228.79,230.96,227.50,228.13,8278835,-0.660,-0.288470,228.8450,2026-01,0
8,NVDA,0.449738,0.550262,1.0,0.2,2026-01-06,190.52,192.17,186.82,187.24,176862604,-3.280,-1.720000,189.1875,2026-01,1
9,NVDA,0.449738,0.550262,1.0,0.2,2026-01-05,191.76,193.63,186.15,188.12,183529733,-3.640,-1.900000,189.9150,2026-01,0


In [ ]:
def calculate_portfolio_metrics(portfolio: pd.DataFrame) -> pd.DataFrame:
    s = portfolio.groupby("date")['close'].sum()
    df = pd.DataFrame(s, columns=['close']).reset_index()
    df['daily_return'] = df['close'].pct_change()
    df.loc[0, 'daily_return'] = 0
    df['cumulative_return'] = (df['close'] - df['close'][0]) / df['close'][0] 

    return df

# s_portfolio_metric = df_weekly_portfolio.groupby("date")['close'].sum()
# df_portfolio_metric = pd.DataFrame(s_portfolio_metric, columns=['close']).reset_index()

In [ ]:
# class Portfolio_Tracker:
#     def __init__(self, portfolio, start_date='2026-01-01'):
#         self.portfolio = portfolio
#         self.start_date = start_date
# total_value = 1000000
# if len(df_weekly_portfolio) != 0:
#     df_weekly_portfolio['asset_allocated'] = total_value * df_weekly_portfolio['weight'] 
#     df_weekly_portfolio.loc[df_weekly_portfolio['week_of_day'] == 0, 'shares_per_stock'] = df_weekly_portfolio['asset_allocated'] / df_weekly_portfolio['close']
    

# df_weekly_portfolio




In [7]:
df_weekly_perf = calculate_portfolio_metrics(df_weekly_portfolio)
df_weekly_perf

NameError: name 'calculate_portfolio_metrics' is not defined

In [6]:
# file_path = 'hist_perf.csv'
# p = DATA_DIR/'processed'/Path(file_path)

# if p.is_file():
#     print('yes')
# else:
#     print('no')
# current_date = datetime.now().date()


2026-01-08


In [ ]:
#create a new function that tracks the historical performance. 
#create the new function by merging the calculate weekly portfolio metrics function
def historical_performance(hist_perf_file_path, df_weekly_perf, curr_date: datetime, start_date = "2026-01-05", init_value = 1000000):
    if hist_perf_file_path.is_file():
        df_hist_perf = pd.read_csv(hist_perf_file_path)
    else: 
        df_hist_perf = pd.DataFrame()
    curr_date_str = str(curr_date)
    df_daily_perf = df_weekly_perf[curr_date_str]
    df_hist_perf = pd.concat([df_hist_perf, df_daily_perf])

    return df_hist_perf